In [ ]:
from itertools import combinations
import pandas as pd
import os

In [ ]:
sum_grid = [
    [-1, 18, -1, -1, -1, -1, 7, -1, -1],
    [-1, -1, -1, -1, 12, -1, -1, -1, -1],
    [-1, -1, 9, -1, -1, -1, -1, 31, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 5, -1, 11, -1, 22, -1, 22, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 9, -1, -1, -1, -1, 19, -1, -1],
    [-1, -1, -1, -1, 14, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 22, -1, -1, -1, -1, 15, -1],
]

# sum_grid = [
#     [0,-1,-1,-1,-1],
#     [-1,-1,9,-1,7],
#     [8,-1,-1,-1,-1],
#     [-1,-1,15,-1,12],
#     [10,-1,-1,-1,-1]
# ]

placed_grid = [
    [False for _ in range(len(sum_grid))] for _ in range(len(sum_grid))
]

number_grid = [
    [-1 for _ in range(len(sum_grid))] for _ in range(len(sum_grid))
]


def print_2d_array_nicely(arr):
    print(pd.DataFrame(arr))


def get_corners(hooks_grid):
    corners = []

    # Find bounds of 0 region
    x_min = len(hooks_grid)
    x_max = 0
    y_min = len(hooks_grid[0])
    y_max = 0

    for i in range(len(hooks_grid)):
        for j in range(len(hooks_grid[0])):
            if hooks_grid[i][j] == 0:
                x_min = min(x_min, i)
                y_min = min(y_min, j)
                x_max = max(x_max, i)
                y_max = max(y_max, j)

    # Append corners
    corners.append((x_min, y_min))
    corners.append((x_max, y_min))
    corners.append((x_min, y_max))
    corners.append((x_max, y_max))

    return corners


def place_hook(hooks_grid, hook_size, x, y):
    above = [hooks_grid[i][y] for i in range(x)]
    below = [hooks_grid[i][y] for i in range(x + 1, len(hooks_grid))]
    right = hooks_grid[x][y + 1 :]
    left = hooks_grid[x][:y]

    if all(element == 0 for element in above):
        for i in range(x):
            hooks_grid[i][y] = hook_size

    if all(element == 0 for element in below):
        for i in range(x + 1, len(hooks_grid)):
            hooks_grid[i][y] = hook_size

    if all(element == 0 for element in right):
        for j in range(y + 1, len(hooks_grid[x])):
            hooks_grid[x][j] = hook_size

    if all(element == 0 for element in left):
        for j in range(y):
            hooks_grid[x][j] = hook_size
    hooks_grid[x][y] = hook_size

    return hooks_grid


def generate_hooks_grids(hooks_grid, current_hook_size=9):
    if current_hook_size == 0:
        yield hooks_grid
        return

    corners = get_corners(hooks_grid)

    for corner in corners:
        # Make a copy of the grid to try placing the current hook
        new_grid = [row[:] for row in hooks_grid]
        new_grid = place_hook(new_grid, current_hook_size, *corner)
        yield from generate_hooks_grids(new_grid, current_hook_size - 1)


initial_hooks_grid = [
    [0 for _ in range(len(sum_grid))] for _ in range(len(sum_grid))
]

for possible_hooks_grid in generate_hooks_grids(initial_hooks_grid):
    print(possible_hooks_grid)
    break

In [ ]:
"""
now, place numbers per the hooks_grid and check if good
"""


def is_cell_bad(number_grid, sum_grid, placed_grid, cell):
    x, y = cell
    rows, cols = len(sum_grid), len(sum_grid[0])

    def is_valid(r, c):
        return 0 <= r < rows and 0 <= c < cols and number_grid[r][c] != 0

    # check for sum grid
    if sum_grid[x][y] != -1:
        total, didnt_break = 0, True
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            if is_valid(x + dx, y + dy):
                if not placed_grid[x + dx][y + dy]:
                    didnt_break = False
                    break
                total += number_grid[x + dx][y + dy]
        if didnt_break and total != sum_grid[x][y]:
            return True

    squares = []

    # Check for valid starting positions to avoid out-of-bounds errors
    for i in range(max(0, x - 1), min(rows - 1, x)):
        for j in range(max(0, y - 1), min(cols - 1, y)):
            # Check if all elements in the square exist within the matrix bounds
            if i + 1 < rows and j + 1 < cols:
                squares.append(
                    [[i, j], [i, j + 1], [i + 1, j], [i + 1, j + 1]]
                )

    def bad_square_found(square):
        return (
            all(placed_grid[i][j] for i, j in square)
            and list(map(lambda x: number_grid[x[0]][x[1]], square)).count(-1)
            == 0
        )

    if any(bad_square_found(square) for square in squares):
        return True
    return False


def place_numbers(
    number_grid,
    hooks_grid,
    sum_grid,
    placed_grid,
    current_number=9,
    available_hooks=set(range(1, 10)),
):
    if current_number == 0:
        # Base case: all numbers placed, yield the current grid
        yield number_grid
        return

    for hook_value in sorted(available_hooks, reverse=True):
        hook_cells = [
            (i, j)
            for i in range(len(hooks_grid))
            for j in range(len(hooks_grid[0]))
            if hooks_grid[i][j] == hook_value
        ]

        if len(hook_cells) >= current_number:
            for placement in combinations(hook_cells, current_number):
                # prune with 2x2 requirement and starting value requirement

                trial_number_grid = [row[:] for row in number_grid]
                trial_placed_grid = [row[:] for row in placed_grid]
                for cell in placement:
                    x, y = cell
                    trial_number_grid[x][y] = current_number
                    trial_placed_grid[x][y] = True

                should_continue = False
                for cell in placement:
                    if is_cell_bad(number_grid, sum_grid, placed_grid, cell):
                        should_continue = True
                if should_continue:
                    continue

                new_available_hooks = available_hooks.copy()
                new_available_hooks.remove(hook_value)

                yield from place_numbers(
                    trial_number_grid,
                    hooks_grid,
                    sum_grid,
                    trial_placed_grid,
                    current_number - 1,
                    new_available_hooks,
                )


def validate_number_grid(number_grid, sum_grid):
    rows, cols = len(number_grid), len(number_grid[0])

    # Sum check
    for r in range(rows):
        for c in range(cols):
            if sum_grid[r][c] != -1:
                total = 0
                for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                    total += number_grid[r + dr][c + dc]
                if total != sum_grid[r][c]:
                    return False

    # empty cell check
    for r in range(rows - 1):
        for c in range(cols - 1):
            # Check the 2x2 square
            if all(
                number_grid[r][c] != -1
                or number_grid[r + 1][c] != -1
                or number_grid[r][c + 1] != -1
                or number_grid[r + 1][c + 1] != -1
            ):
                return False

    # check for single island
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    islands = 0

    def is_valid(r, c):
        return 0 <= r < rows and 0 <= c < cols and number_grid[r][c] != 0

    def dfs(r, c):
        if not is_valid(r, c) or visited[r][c] or number_grid[r][c] == -1:
            return
        visited[r][c] = True
        dfs(r + 1, c)
        dfs(r - 1, c)
        dfs(r, c + 1)
        dfs(r, c - 1)

    for r in range(rows):
        for c in range(cols):
            if number_grid[r][c] != -1:
                islands += 1
                dfs(r, c)

    return islands == 1

In [80]:
for possible_hooks_grid in generate_hooks_grids(initial_hooks_grid):
    for possible_filled_number_grid in place_numbers(
        number_grid, possible_hooks_grid, sum_grid, placed_grid
    ):
        if validate_number_grid(possible_filled_number_grid, sum_grid):
            with open(os.path.join(os.getcwd(), "output.txt"), "a") as f:
                f.write(
                    pd.DataFrame(possible_hooks_grid).to_string(index=False)
                )
                f.write("\n\n")
                f.write(
                    pd.DataFrame(possible_filled_number_grid).to_string(
                        index=False
                    )
                )
                f.write("\n\n\n")

KeyboardInterrupt: 